In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import time
from scipy import stats
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read in data

## combine prev and label

In [3]:
sessions_test = pd.read_csv('/content/drive/MyDrive/SocialNetworkAnalysis-2024Spring/Final Project/data/sessions_test_task1_phase1.csv')

In [4]:
sessions_test.head()

,prev_items,locale
0,['B08V12CT4C' 'B08V1KXBQD' 'B01BVG1XJS' 'B09VC...,DE
1,['B00R9R5ND6' 'B00R9RZ9ZS' 'B00R9RZ9ZS'],DE
2,['B07YSRXJD3' 'B07G7Q5N6G' 'B08C9Q7QVK' 'B07G7...,DE
3,['B08KQBYV43' '3955350843' '3955350843' '39553...,DE
4,['B09FPTCWMC' 'B09FPTQP68' 'B08HMRY8NG' 'B08TB...,DE


In [5]:
labels = pd.read_csv("/content/drive/MyDrive/SocialNetworkAnalysis-2024Spring/Final Project/data/gt_task1.csv")
labels.head()

,next_item,locale
0,B09KG6VDF2,DE
1,B004ZXMV4Q,DE
2,B09KY376VZ,DE
3,B0829LZFT1,DE
4,B09J8V9RQQ,DE


In [6]:
sessions_test['next_item'] = labels['next_item']

In [7]:
sessions_test.head()

,prev_items,locale,next_item
0,['B08V12CT4C' 'B08V1KXBQD' 'B01BVG1XJS' 'B09VC...,DE,B09KG6VDF2
1,['B00R9R5ND6' 'B00R9RZ9ZS' 'B00R9RZ9ZS'],DE,B004ZXMV4Q
2,['B07YSRXJD3' 'B07G7Q5N6G' 'B08C9Q7QVK' 'B07G7...,DE,B09KY376VZ
3,['B08KQBYV43' '3955350843' '3955350843' '39553...,DE,B0829LZFT1
4,['B09FPTCWMC' 'B09FPTQP68' 'B08HMRY8NG' 'B08TB...,DE,B09J8V9RQQ


## read

In [8]:
sessions_valid = pd.read_csv("/content/drive/MyDrive/SocialNetworkAnalysis-2024Spring/Final Project/data/sessions_valid.csv")
sessions_valid_trimmed = pd.read_csv("/content/drive/MyDrive/SocialNetworkAnalysis-2024Spring/Final Project/data/sessions_valid_trimmed.csv")
sessions_valid_trimmed_5 = pd.read_csv("/content/drive/MyDrive/SocialNetworkAnalysis-2024Spring/Final Project/data/sessions_valid_trimmed_5.csv")

In [9]:
valid_products_df = pd.read_csv("/content/drive/MyDrive/SocialNetworkAnalysis-2024Spring/Final Project/data/valid_products_df.csv")

In [10]:
sessions_valid_trimmed_5

,five_prev_items,next_item,valid
0,"['0', '0', 'B00LW1APOC', 'B00OI6NQUI', 'B09HLD...",B07H54NZ3K,True
1,"['0', '0', '0', 'B079DZRVDF', 'B079FD3MGB']",B09PKCR4CX,False
2,"['0', '0', '0', '0', 'B00D8CXYXC']",B01MYGEMXU,True
3,"['0', '0', 'B07BR8JQWC', 'B07BR8JQWC', 'B002GJ...",B002I4DJOA,True
4,"['B08DM17RDG', 'B08DM1BC3F', 'B09Q6KCGC4', 'B0...",B0849WB17J,False
...,...,...,...
276146,"['B06Y5YQMHZ', 'B0036162JI', 'B0036162JI', 'B0...",B09WDQ2C89,True
276147,"['0', '0', '0', '0', 'B089DGVVXQ']",B086435TB4,False
276148,"['B09J4TYGF3', 'B0B58WWP97', 'B098Q3CCC6', 'B0...",B09YR7BP3R,False
276149,"['0', 'B07L9SJ214', 'B08M4G1QFS', 'B07L9SJ214'...",B07L9N986G,True


In [11]:
valid_products_df

,valid_products
0,B072R3PR8Q
1,B0756XZGF2
2,B00009W9K1
3,B09BNK9YM3
4,B07RJXFX87
...,...
121925,B00H6J4G9O
121926,B09S7GWQ8R
121927,B0002DV6RQ
121928,B0046ZRYVS


# Data Cleaning-Sessions

## Filter UK

In [43]:
sessions_uk = sessions_test.loc[sessions_test['locale']=='UK']
sessions_uk.head()

,prev_items,locale,next_item
201035,['B007MO0FIO' 'B09QXPN2TL' 'B005PA3I8G' 'B003Y...,UK,B0030G0FG0
201036,['B082P6M3HC' 'B082BHJ9G8' 'B0797QCXS6' 'B082B...,UK,B089Y246KV
201037,['B0894HWG1H' 'B07S79G11S' 'B092VCHVS7' 'B09DC...,UK,B08ZL8G35H
201038,['B00FLTE8OE' 'B07CRNCKMR' 'B0784T65RK'],UK,B09VPPX599
201039,['B0B5XPGRXD' 'B0BHSXQLSX' 'B0BJKJZJNM'],UK,B09Z2V97N5


In [44]:
sessions_uk.info()

<class 'pandas.core.frame.DataFrame'>
Index: 115936 entries, 201035 to 316970
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   prev_items  115936 non-null  object
 1   locale      115936 non-null  object
 2   next_item   115936 non-null  object
dtypes: object(3)
memory usage: 3.5+ MB


## Seperate prev_items

In [45]:
prev_items_sep = list(map(lambda row: [i.replace("'", "") for i in row.strip('[]').split()], sessions_uk['prev_items']))
sessions_uk['prev_items_sep'] = prev_items_sep
sessions_uk.head()

<ipython-input-45-ca125790ead5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sessions_uk['prev_items_sep'] = prev_items_sep


,prev_items,locale,next_item,prev_items_sep
201035,['B007MO0FIO' 'B09QXPN2TL' 'B005PA3I8G' 'B003Y...,UK,B0030G0FG0,"[B007MO0FIO, B09QXPN2TL, B005PA3I8G, B003Y7MS6..."
201036,['B082P6M3HC' 'B082BHJ9G8' 'B0797QCXS6' 'B082B...,UK,B089Y246KV,"[B082P6M3HC, B082BHJ9G8, B0797QCXS6, B082BGVKT..."
201037,['B0894HWG1H' 'B07S79G11S' 'B092VCHVS7' 'B09DC...,UK,B08ZL8G35H,"[B0894HWG1H, B07S79G11S, B092VCHVS7, B09DCPQG5..."
201038,['B00FLTE8OE' 'B07CRNCKMR' 'B0784T65RK'],UK,B09VPPX599,"[B00FLTE8OE, B07CRNCKMR, B0784T65RK]"
201039,['B0B5XPGRXD' 'B0BHSXQLSX' 'B0BJKJZJNM'],UK,B09Z2V97N5,"[B0B5XPGRXD, B0BHSXQLSX, B0BJKJZJNM]"


In [46]:
sessions_uk_new = sessions_uk.reset_index()[['prev_items_sep', 'next_item']]
sessions_uk_new.head()

,prev_items_sep,next_item
0,"[B007MO0FIO, B09QXPN2TL, B005PA3I8G, B003Y7MS6...",B0030G0FG0
1,"[B082P6M3HC, B082BHJ9G8, B0797QCXS6, B082BGVKT...",B089Y246KV
2,"[B0894HWG1H, B07S79G11S, B092VCHVS7, B09DCPQG5...",B08ZL8G35H
3,"[B00FLTE8OE, B07CRNCKMR, B0784T65RK]",B09VPPX599
4,"[B0B5XPGRXD, B0BHSXQLSX, B0BJKJZJNM]",B09Z2V97N5


# Feature Engineering-Sessions

## Create column prev_items_len

In [47]:
prev_len = list(map(lambda row: len(row), sessions_uk_new['prev_items_sep']))
sessions_uk_new['prev_items_len'] = prev_len
sessions_uk_new.head()

,prev_items_sep,next_item,prev_items_len
0,"[B007MO0FIO, B09QXPN2TL, B005PA3I8G, B003Y7MS6...",B0030G0FG0,7
1,"[B082P6M3HC, B082BHJ9G8, B0797QCXS6, B082BGVKT...",B089Y246KV,6
2,"[B0894HWG1H, B07S79G11S, B092VCHVS7, B09DCPQG5...",B08ZL8G35H,8
3,"[B00FLTE8OE, B07CRNCKMR, B0784T65RK]",B09VPPX599,3
4,"[B0B5XPGRXD, B0BHSXQLSX, B0BJKJZJNM]",B09Z2V97N5,3


In [23]:
#sessions_uk_new.to_csv('/content/drive/MyDrive/SocialNetworkAnalysis-2024Spring/Final Project/data/sessions_uk_withPrevItemLength.csv', index=False)

In [22]:
# prev_items_counts = sessions_uk_new['prev_items_len'].value_counts()
# prev_items_counts

# We might want to think about:
# 1. whether the sessions that has really long prev_items are worth to consider
#    If we want to plot them, the graph will be really 'long' if we consider one
#    session as a whole.
#    If we only consider the previous item and the next item, this will not be a
#    a problem.

## Filter out invalid sessions

In [48]:
def trim_list(l, valid_products):
    i = -1
    while abs(i) < len(l) and l[i] in valid_products:
        i -= 1
    i += 1

    if i == 0:
        return []
    else:
        return l[i:]

def trim_df(df, valid_products, start_index, end_index):
  for i in range(start_index, end_index+1):
    df.at[i, "five_prev_items"] = trim_list(df.loc[i, "five_prev_items"], valid_products)

In [49]:
valid_products = list(valid_products_df['valid_products'])
valid_products[0:5]

['B072R3PR8Q', 'B0756XZGF2', 'B00009W9K1', 'B09BNK9YM3', 'B07RJXFX87']

In [50]:
sessions_uk_new["valid"] = [item in valid_products for item in sessions_uk_new["next_item"]]
sessions_uk_new

# None of the next_item is in out valid products list

,prev_items_sep,next_item,prev_items_len,valid
0,"[B007MO0FIO, B09QXPN2TL, B005PA3I8G, B003Y7MS6...",B0030G0FG0,7,True
1,"[B082P6M3HC, B082BHJ9G8, B0797QCXS6, B082BGVKT...",B089Y246KV,6,True
2,"[B0894HWG1H, B07S79G11S, B092VCHVS7, B09DCPQG5...",B08ZL8G35H,8,False
3,"[B00FLTE8OE, B07CRNCKMR, B0784T65RK]",B09VPPX599,3,False
4,"[B0B5XPGRXD, B0BHSXQLSX, B0BJKJZJNM]",B09Z2V97N5,3,True
...,...,...,...,...
115931,"[B077SZ2C3Y, B0B14M3VZX]",B08X9L5RGD,2,True
115932,"[B08KFHDPY9, B0851KTSRZ, B08KFHDPY9, B0851KTSR...",B09CPNS7XV,5,True
115933,"[B07PY1N81F, B07Q1Z8SQN, B07PY1N81F, B07Q1Z8SQ...",B09HZSRJWW,6,False
115934,"[B01MCQMORK, B09JYZ325W]",B09X2G8GH7,2,False


In [51]:
sessions_uk_new = sessions_uk_new[sessions_uk_new['valid']==True].reset_index(drop=True)
sessions_uk_new

,prev_items_sep,next_item,prev_items_len,valid
0,"[B007MO0FIO, B09QXPN2TL, B005PA3I8G, B003Y7MS6...",B0030G0FG0,7,True
1,"[B082P6M3HC, B082BHJ9G8, B0797QCXS6, B082BGVKT...",B089Y246KV,6,True
2,"[B0B5XPGRXD, B0BHSXQLSX, B0BJKJZJNM]",B09Z2V97N5,3,True
3,"[B01EY8LJTE, B08CDB4YZG, B06ZZPRHNH, B07W8GJ8F...",B00PIQXWVK,5,True
4,"[B097DWZL5S, B07C5HVQSL, B09ZLJF49P, B09ZLJMW4...",B073FG6KMC,9,True
...,...,...,...,...
46477,"[B0BJ18T3BX, B08KGTW3CV]",B08KGV6XTX,2,True
46478,"[B01117UJ8S, B00C71EBXC]",B09B7R9XN8,2,True
46479,"[B09BZR9JFG, B08VNPVFWW, B09BZR9JFG, B08VNPVFW...",B07YF9Y74S,13,True
46480,"[B077SZ2C3Y, B0B14M3VZX]",B08X9L5RGD,2,True


## keep last five item in each test session

In [52]:
prev5 = []

for i in range(len(sessions_uk_new)):
    if sessions_uk_new.loc[i, 'prev_items_len'] > 5:
        prev5.append(sessions_uk_new.loc[i, 'prev_items_sep'][-5:])
    else:
        lis = []
        for _ in range(5-sessions_uk_new.loc[i, 'prev_items_len']):
            lis.append("0")
        lis.extend(sessions_uk_new.loc[i, 'prev_items_sep'])
        prev5.append(lis)

In [53]:
sessions_uk_new["five_prev_items"] = prev5
sessions_uk_new

,prev_items_sep,next_item,prev_items_len,valid,five_prev_items
0,"[B007MO0FIO, B09QXPN2TL, B005PA3I8G, B003Y7MS6...",B0030G0FG0,7,True,"[B005PA3I8G, B003Y7MS6U, B005PA3I8G, B003Y7MS6..."
1,"[B082P6M3HC, B082BHJ9G8, B0797QCXS6, B082BGVKT...",B089Y246KV,6,True,"[B082BHJ9G8, B0797QCXS6, B082BGVKTR, B082BHF6Z..."
2,"[B0B5XPGRXD, B0BHSXQLSX, B0BJKJZJNM]",B09Z2V97N5,3,True,"[0, 0, B0B5XPGRXD, B0BHSXQLSX, B0BJKJZJNM]"
3,"[B01EY8LJTE, B08CDB4YZG, B06ZZPRHNH, B07W8GJ8F...",B00PIQXWVK,5,True,"[B01EY8LJTE, B08CDB4YZG, B06ZZPRHNH, B07W8GJ8F..."
4,"[B097DWZL5S, B07C5HVQSL, B09ZLJF49P, B09ZLJMW4...",B073FG6KMC,9,True,"[B09ZLHWC1H, B073FH7QQ1, B073FH44VC, B073FMZNP..."
...,...,...,...,...,...
46477,"[B0BJ18T3BX, B08KGTW3CV]",B08KGV6XTX,2,True,"[0, 0, 0, B0BJ18T3BX, B08KGTW3CV]"
46478,"[B01117UJ8S, B00C71EBXC]",B09B7R9XN8,2,True,"[0, 0, 0, B01117UJ8S, B00C71EBXC]"
46479,"[B09BZR9JFG, B08VNPVFWW, B09BZR9JFG, B08VNPVFW...",B07YF9Y74S,13,True,"[B08VNPVFWW, B08VNPVFWW, B08XBZYXQS, B08XC4JH7..."
46480,"[B077SZ2C3Y, B0B14M3VZX]",B08X9L5RGD,2,True,"[0, 0, 0, B077SZ2C3Y, B0B14M3VZX]"


In [54]:
final_sessions = sessions_uk_new[['five_prev_items', 'next_item']]
final_sessions

,five_prev_items,next_item
0,"[B005PA3I8G, B003Y7MS6U, B005PA3I8G, B003Y7MS6...",B0030G0FG0
1,"[B082BHJ9G8, B0797QCXS6, B082BGVKTR, B082BHF6Z...",B089Y246KV
2,"[0, 0, B0B5XPGRXD, B0BHSXQLSX, B0BJKJZJNM]",B09Z2V97N5
3,"[B01EY8LJTE, B08CDB4YZG, B06ZZPRHNH, B07W8GJ8F...",B00PIQXWVK
4,"[B09ZLHWC1H, B073FH7QQ1, B073FH44VC, B073FMZNP...",B073FG6KMC
...,...,...
46477,"[0, 0, 0, B0BJ18T3BX, B08KGTW3CV]",B08KGV6XTX
46478,"[0, 0, 0, B01117UJ8S, B00C71EBXC]",B09B7R9XN8
46479,"[B08VNPVFWW, B08VNPVFWW, B08XBZYXQS, B08XC4JH7...",B07YF9Y74S
46480,"[0, 0, 0, B077SZ2C3Y, B0B14M3VZX]",B08X9L5RGD


In [25]:
sessions_valid_trimmed_5

,five_prev_items,next_item,valid
0,"['0', '0', 'B00LW1APOC', 'B00OI6NQUI', 'B09HLD...",B07H54NZ3K,True
1,"['0', '0', '0', 'B079DZRVDF', 'B079FD3MGB']",B09PKCR4CX,False
2,"['0', '0', '0', '0', 'B00D8CXYXC']",B01MYGEMXU,True
3,"['0', '0', 'B07BR8JQWC', 'B07BR8JQWC', 'B002GJ...",B002I4DJOA,True
4,"['B08DM17RDG', 'B08DM1BC3F', 'B09Q6KCGC4', 'B0...",B0849WB17J,False
...,...,...,...
276146,"['B06Y5YQMHZ', 'B0036162JI', 'B0036162JI', 'B0...",B09WDQ2C89,True
276147,"['0', '0', '0', '0', 'B089DGVVXQ']",B086435TB4,False
276148,"['B09J4TYGF3', 'B0B58WWP97', 'B098Q3CCC6', 'B0...",B09YR7BP3R,False
276149,"['0', 'B07L9SJ214', 'B08M4G1QFS', 'B07L9SJ214'...",B07L9N986G,True


In [55]:
trim_df(final_sessions, valid_products, 0, len(sessions_uk_new)-1)

In [56]:
final_sessions

,five_prev_items,next_item
0,"[B003Y7MS6U, B005PA3I8G, B003Y7MS6U, B007MO0FIO]",B0030G0FG0
1,"[B0797QCXS6, B082BGVKTR, B082BHF6ZZ, B082P6M3HC]",B089Y246KV
2,[],B09Z2V97N5
3,[B01DY36K5I],B00PIQXWVK
4,"[B073FH7QQ1, B073FH44VC, B073FMZNPF, B09GPSJSPH]",B073FG6KMC
...,...,...
46477,[B08KGTW3CV],B08KGV6XTX
46478,"[0, 0, B01117UJ8S, B00C71EBXC]",B09B7R9XN8
46479,[B08CN3G4N9],B07YF9Y74S
46480,[B0B14M3VZX],B08X9L5RGD


In [59]:
blank_lists = 0
for i in range(len(final_sessions)):
  if not final_sessions.loc[i, "five_prev_items"]:
    blank_lists += 1

blank_lists

14096

In [61]:
len(final_sessions) - blank_lists

32386

In [63]:
sessions_valid = final_sessions.copy()

prev_len_trimmed = list(map(lambda row: len(row), sessions_valid['five_prev_items']))
sessions_valid['prev_items_len'] = prev_len_trimmed
sessions_valid


,five_prev_items,next_item,prev_items_len
0,"[B003Y7MS6U, B005PA3I8G, B003Y7MS6U, B007MO0FIO]",B0030G0FG0,4
1,"[B0797QCXS6, B082BGVKTR, B082BHF6ZZ, B082P6M3HC]",B089Y246KV,4
2,[],B09Z2V97N5,0
3,[B01DY36K5I],B00PIQXWVK,1
4,"[B073FH7QQ1, B073FH44VC, B073FMZNPF, B09GPSJSPH]",B073FG6KMC,4
...,...,...,...
46477,[B08KGTW3CV],B08KGV6XTX,1
46478,"[0, 0, B01117UJ8S, B00C71EBXC]",B09B7R9XN8,4
46479,[B08CN3G4N9],B07YF9Y74S,1
46480,[B0B14M3VZX],B08X9L5RGD,1


In [74]:
sessions_valid_trimmed = sessions_valid[sessions_valid["prev_items_len"] > 0].reset_index(drop=True)
sessions_valid_trimmed

,five_prev_items,next_item,prev_items_len
0,"[B003Y7MS6U, B005PA3I8G, B003Y7MS6U, B007MO0FIO]",B0030G0FG0,4
1,"[B0797QCXS6, B082BGVKTR, B082BHF6ZZ, B082P6M3HC]",B089Y246KV,4
2,[B01DY36K5I],B00PIQXWVK,1
3,"[B073FH7QQ1, B073FH44VC, B073FMZNPF, B09GPSJSPH]",B073FG6KMC,4
4,"[B089ZPJXWZ, B08BL793R1, B089ZPJXWZ, B08BL793R1]",B0BC8Q6J2B,4
...,...,...,...
32381,[B08KGTW3CV],B08KGV6XTX,1
32382,"[0, 0, B01117UJ8S, B00C71EBXC]",B09B7R9XN8,4
32383,[B08CN3G4N9],B07YF9Y74S,1
32384,[B0B14M3VZX],B08X9L5RGD,1


In [75]:
prev5 = []

for i in range(len(sessions_valid_trimmed)):
    if sessions_valid_trimmed.loc[i, 'prev_items_len'] > 5:
        prev5.append(sessions_valid_trimmed.loc[i, 'five_prev_items'][-5:])
    else:
        lis = []
        for _ in range(5-sessions_valid_trimmed.loc[i, 'prev_items_len']):
            lis.append("0")
        lis.extend(sessions_valid_trimmed.loc[i, 'five_prev_items'])
        prev5.append(lis)

In [76]:
sessions_valid_trimmed["five_prev_items"] = prev5
sessions_valid_trimmed = sessions_valid_trimmed.drop(columns="prev_items_len")
sessions_valid_trimmed

,five_prev_items,next_item
0,"[0, B003Y7MS6U, B005PA3I8G, B003Y7MS6U, B007MO...",B0030G0FG0
1,"[0, B0797QCXS6, B082BGVKTR, B082BHF6ZZ, B082P6...",B089Y246KV
2,"[0, 0, 0, 0, B01DY36K5I]",B00PIQXWVK
3,"[0, B073FH7QQ1, B073FH44VC, B073FMZNPF, B09GPS...",B073FG6KMC
4,"[0, B089ZPJXWZ, B08BL793R1, B089ZPJXWZ, B08BL7...",B0BC8Q6J2B
...,...,...
32381,"[0, 0, 0, 0, B08KGTW3CV]",B08KGV6XTX
32382,"[0, 0, 0, B01117UJ8S, B00C71EBXC]",B09B7R9XN8
32383,"[0, 0, 0, 0, B08CN3G4N9]",B07YF9Y74S
32384,"[0, 0, 0, 0, B0B14M3VZX]",B08X9L5RGD


In [80]:
sessions_valid_trimmed.to_csv("/content/drive/MyDrive/SocialNetworkAnalysis-2024Spring/Final Project/data/test_sessions_final.csv", index=False)